In [1]:
import sys, os, inspect

# add common to path
from pathlib import Path
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
common_path = str(Path(currentdir).parent.parent)
sys.path.append( common_path )

from common.nlp.vocab import Vocab
from common.nlp.data_loader import N21TextData
from common.nlp.converter import N21Converter

from dataset import SentimentDataset
from dataset import load_data
from common.ml.hparams import HParams

import numpy as np
import copy 
import time 
import tensorflow as tf
from tensorflow.python.ops import rnn
from tensorflow.contrib.layers.python.layers import linear

from common.ml.tf.deploy import freeze_graph

print( "Tensorflow Version : ", tf.__version__)

Tensorflow Version :  1.9.0


In [2]:
sys.path

['',
 'C:\\Users\\juni\\Anaconda3\\python36.zip',
 'C:\\Users\\juni\\Anaconda3\\DLLs',
 'C:\\Users\\juni\\Anaconda3\\lib',
 'C:\\Users\\juni\\Anaconda3',
 'C:\\Users\\juni\\AppData\\Roaming\\Python\\Python36\\site-packages',
 'C:\\Users\\juni\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\juni\\Anaconda3\\lib\\site-packages\\Babel-2.5.0-py3.6.egg',
 'C:\\Users\\juni\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\juni\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\juni\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\juni\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\juni\\.ipython',
 'C:\\Users\\juni\\pythonNlp\\course\\nlp']

In [2]:
class SentimentAnalysis():
    def __init__(self, hps, mode="train"):
        self.hps = hps
        self.x = tf.placeholder(tf.int32, [None, hps.num_steps], name="pl_tokens")#None: batch_size를 모르기 때문에 알아서 하도록 함
        self.y = tf.placeholder(tf.int32, [None], name="pl_target")#placeholder: 다른 텐서를 할당하기 위해사용, 이름도 써주는게 좋음
        self.w = tf.placeholder(tf.float32, [None, hps.num_steps], name="pl_weight")
        self.keep_prob = tf.placeholder(tf.float32, [], name="pl_keep_prob")

        ### 4 blocks ###
        # 1) embedding
        # 2) dropout on input embedding
        # 3) sentence encoding using rnn
        # 4) encoding to output classes
        # 5) loss calcaulation

        def _embedding(x):
            # character embedding 
            #print("-"*100)
            #print("Implement function '{}'".format(_embedding.__name__))
            #print("Keywords")
            #print("\t - tf.initializers.variance_scaling")
            #print("\t - tf.get_variable")
            #print("\t - tf.nn.embedding_lookup")
            #print("\t - tf.unstack")

            #print('Input : Tensor("model/pl_tokens:0", shape=(?, 128), dtype=int32)')

            
            #print("Return: a list of <tf.Tensor shape=(?, 50) dtype=float32>")
            #print("        len(a_list) = 128")
            shape = [hps.vocab_size,hps.emb_size]
            initializer = tf.initializers.variance_scaling(distribution="uniform",dtype=tf.float32)#가중치 벡터의 shape에 맞게 조정
            emb_mat = tf.get_variable('emb',shape,initializer=initializer,dtype=tf.float32)#입력한 파라미터에 맞는 데이터를 생성하거나 가져옴
            input_emb = tf.nn.embedding_lookup(emb_mat,x)#embedding tensor리스트에서 id를 검색함
            
            step_inputs = tf.unstack(input_emb,axis=1) # split input_emb -> num_steps, tf.unstack: 랭크를 1줄임
            return step_inputs

        def _sequence_dropout(step_inputs, keep_prob):
            # apply dropout to each input
            # input : a list of input tensor which shape is [None, input_dim]
            print("-"*100)
            with tf.name_scope('sequence_dropout') as scope:
                #print("Implement step_outputs")
                #print("Keywords")
                #print("\t - tf.nn.dropout")
                step_outputs=[]
                for t,input in enumerate(step_inputs):
                    step_outputs.append(tf.nn.dropout(input,keep_prob))



            #print("Return: a list of <tf.Tensor shape=(?, 50) dtype=float32>")
            return step_outputs

        def sequence_encoding_n21_rnn(step_inputs, cell_size, scope_name):
            # rnn based N21 encoding (GRU)
            #print("-"*100)
            #print("Implement function '{}'".format(sequence_encoding_n21_rnn.__name__))
            #print('Input : a list of <tf.Tensor shape=(?, 50), dtype=float32>')
            #print("Keywords")
            #print("\t - tf.contrib.rnn.GRUCell")
            #print("\t - tf.contrib.rnn.static_rnn")

            #print("Return: a list of <tf.Tensor, shape=(?, 100)>")
            step_inputs=list(reversed(step_inputs))#뒤 쪽에 있는 패딩심볼의 악영향을 줄이기 위해 뒤에서 부터 인코딩
            f_rnn_cell=tf.contrib.rnn.GRUCell(cell_size,reuse=None)
            _inputs=tf.stack(step_inputs,axis=1)
            step_outputs, final_state=tf.contrib.rnn.static_rnn(f_rnn_cell,
                                                               step_inputs,
                                                               dtype=tf.float32,
                                                               scope=scope_name)#각 step별 output과 마지막 히든레이어를 결과로
            out = step_outputs[-1]            
            return out

        def _to_class(input, num_class):
            #print("-"*100)
            #print("Implement function '{}'".format(_to_class.__name__))
            #print('Input : tf.Tensor, shape=(?, 100), dtype=float32')
            #print("Keywords")
            #print("\t - tensorflow.contrib.layers.python.layers.linear")

            #print("Return: tf.Tensor, shape=(?, 4), dtype=float32")
            out = linear(input,num_class,scope="Rnn2Sentiment") # should be implemented
            return out

        def _loss(out, ref):
            # out : [batch_size, num_class] float - unscaled logits
            # ref : [batch_size] integer
            # calculate loss function using cross-entropy
            #print("-"*100)
            #print('Input out: tf.Tensor, shape=(?, 4)')
            #print('Input ref: tf.Tensor("model/pl_target:0", shape=(?,), dtype=int32)')

            #print("Keywords")
            #print("\t - tf.nn.sparse_softmax_cross_entropy_with_logits")
            #print("\t - tf.reduce_mean")
            
            #print("Return: tf.Tensor, shape=(), dtype=float32")
            batch_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=out,labels=ref,name="sentiment_loss")
            loss =  tf.reduce_mean(batch_loss)# [batch_size,emb_dim=30]
            return loss
        
        seq_length    = tf.reduce_sum(self.w, 1) # [batch_size]

        step_inputs   = _embedding(self.x) # [batch_size,emb_dim=50]
        step_inputs   = _sequence_dropout(step_inputs, self.keep_prob)
        sent_encoding = sequence_encoding_n21_rnn(step_inputs, hps.enc_dim, scope_name="encoder")#sentence encoding
        out           = _to_class(sent_encoding, hps.num_target_class)
        loss          = _loss(out, self.y) 

        if loss is None: 
            print("All functions should be implemented!")
            import sys; sys.exit()


        out_probs     = tf.nn.softmax(out, name="out_probs")
        out_pred      = tf.argmax(out_probs, 1, name="out_pred")

        self.loss      = loss
        self.out_probs = out_probs#[batch_size, num_class]
        self.out_pred  = out_pred#[batch_size]

        self.global_step = tf.get_variable("global_step", [], tf.int32, initializer=tf.zeros_initializer, trainable=False)

        if mode == "train":#backpropagation
            optimizer       = tf.train.AdamOptimizer(hps.learning_rate)#자연어에서는 주로 adam사용
            self.train_op   = optimizer.minimize(self.loss, global_step=self.global_step)
        else:
            self.train_op = tf.no_op()


    @staticmethod
    def get_default_hparams():
        return HParams(
            learning_rate     = 0.001,
            keep_prob         = 0.5,#문제의 손상율
        )

In [3]:
def train(train_id_data, num_vocabs, num_taget_class):
    #
    # train sentiment analysis using given train_id_data
    #
    max_epoch = 3000
    model_dir = "./trained_models"
    hps = SentimentAnalysis.get_default_hparams()
    hps.update(
                    batch_size= 100,
                    num_steps = 128,#몇 개의 char가 들어오나? 즉 input
                    emb_size  = 50,#조금 작은 값임
                    enc_dim   = 100,#마지막 시퀀스 인코더의 dimension
                    vocab_size=num_vocabs,
                    num_target_class=num_taget_class
               )

    with tf.variable_scope("model"):#해당 스코프안에서 정의되는 변수는 해당범위에만 한정되어짐
        model = SentimentAnalysis(hps, "train")

    sv = tf.train.Supervisor(is_chief=True,#프로그래밍 패턴중 하나, sv안에 세션을 담으면 알아서 다 해준다고 함
                             logdir=model_dir,
                             summary_op=None,  
                             global_step=model.global_step)

    # tf assign compatible operators for gpu and cpu 
    tf_config = tf.ConfigProto(allow_soft_placement=True)#cpu, gpu가 문제없이 돌아갈 수 있도록 해주는 역할

    with sv.managed_session(config=tf_config) as sess:
        local_step       = 0
        prev_global_step = sess.run(model.global_step)

        train_data_set = SentimentDataset(train_id_data, hps.batch_size, hps.num_steps)
        losses = []
        while not sv.should_stop():#미니배치로 실행
            fetches = [model.global_step, model.loss, model.train_op]
            a_batch_data = next( train_data_set.iterator )
            y, x, w = a_batch_data
            fetched = sess.run(fetches, {
                                            model.x: x, 
                                            model.y: y, 
                                            model.w: w,

                                            model.keep_prob: hps.keep_prob,
                                        }
                              )

            local_step += 1

            _global_step = fetched[0]
            _loss        = fetched[1]
            losses.append( _loss )
            if local_step < 10 or local_step % 10 == 0:
                epoch = train_data_set.get_epoch_num()
                print("Epoch = {:3d} Step = {:7d} loss = {:5.3f}".format(epoch, _global_step, np.mean(losses)) )
                _loss = []                
                if epoch >= max_epoch : break 

        print("Training is done.")
    sv.stop()

    # model.out_pred, model.out_probs
    freeze_graph(model_dir, "model/out_pred,model/out_probs", "frozen_graph.tf.pb") ## freeze graph with params to probobuf format


In [4]:
from tensorflow.core.framework import graph_pb2
def predict(token_vocab, target_vocab, sent):
    #os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # force to use cpu only (prediction)
    os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
    os.environ['CUDA_VISIBLE_DEVICES']='0'# force to use gpu only (prediction)
    model_dir = "./trained_models"

    # prepare sentence converting
    # to make raw sentence to id data easily
    in_sent = '{}\t{}'.format('___DUMMY_CLASS___', sent)
    pred_data     = N21TextData(in_sent, mode='sentence')
    pred_id_data  = N21Converter.convert(pred_data, target_vocab, token_vocab)
    pred_data_set = SentimentDataset(pred_id_data, 1, 128)

    #
    a_batch_data = next(pred_data_set.predict_iterator) # a result
    b_sentiment_id, b_token_ids, b_weight = a_batch_data

    # Restore graph
    # note that frozen_graph.tf.pb contains graph definition with parameter values in binary format
    _graph_fn =  os.path.join(model_dir, 'frozen_graph.tf.pb')
    with tf.gfile.GFile(_graph_fn, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)

    with tf.Session(graph=graph) as sess:
        # to check load graph
        #for n in tf.get_default_graph().as_graph_def().node: print(n.name)

        # make interface for input
        pl_token     = graph.get_tensor_by_name('import/model/pl_tokens:0')
        pl_keep_prob = graph.get_tensor_by_name('import/model/pl_keep_prob:0')

        # make interface for output
        out_pred  = graph.get_tensor_by_name('import/model/out_pred:0')
        out_probs = graph.get_tensor_by_name('import/model/out_probs:0')
        

        # predict sentence 
        b_best_pred_index, b_pred_probs = sess.run([out_pred, out_probs], feed_dict={
                                                                                        pl_token : b_token_ids,
                                                                                        pl_keep_prob : 1.0,
                                                                                    }
                                          )

        best_pred_index = b_best_pred_index[0]
        pred_probs = b_pred_probs[0]
        #결과 출력 루틴
        best_target_class = target_vocab.get_symbol(best_pred_index)
        print( pred_probs[best_pred_index] )
        best_prob  = int( pred_probs[best_pred_index] )
        print(best_target_class, best_prob)

In [5]:
train_id_data, token_vocab, target_vocab = load_data()#데이터 로드
num_vocabs       = token_vocab.get_num_tokens()#vocab로드
num_target_class = target_vocab.get_num_targets()#타겟(카테고리) 로드

train(train_id_data, num_vocabs, num_target_class)#학습
predict(token_vocab, target_vocab, "별로에요")
    


----------------------------------------------------------------------------------------------------
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Restoring parameters from ./trained_models\model.ckpt-26873
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path ./trained_models\model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:model/global_step/sec: 0
Epoch =   1 Step =   26874 loss = 0.000
Epoch =   1 Step =   26875 loss = 0.004
Epoch =   1 Step =   26876 loss = 0.003
Epoch =   1 Step =   26877 loss = 0.002
Epoch =   2 Step =   26878 loss = 0.002
Epoch =   2 Step =   26879 loss = 0.002
Epoch =   2 Step =   26880 loss = 0.001
Epoch =   2 Step =   26881 loss = 0.001
Epoch =   2 Step =   26882 loss = 0.001
Epoch =   3 Step =   26883 loss = 0.001
Epoch =   5 Step =   26893 loss = 0.001
Epoch =   7 Step =   26903 

Epoch = 355 Step =   28643 loss = 0.002
Epoch = 357 Step =   28653 loss = 0.002
Epoch = 359 Step =   28663 loss = 0.002
Epoch = 361 Step =   28673 loss = 0.002
Epoch = 363 Step =   28683 loss = 0.002
Epoch = 365 Step =   28693 loss = 0.002
Epoch = 367 Step =   28703 loss = 0.002
Epoch = 369 Step =   28713 loss = 0.002
Epoch = 371 Step =   28723 loss = 0.002
Epoch = 373 Step =   28733 loss = 0.002
Epoch = 375 Step =   28743 loss = 0.002
Epoch = 377 Step =   28753 loss = 0.002
Epoch = 379 Step =   28763 loss = 0.002
Epoch = 381 Step =   28773 loss = 0.002
Epoch = 383 Step =   28783 loss = 0.002
Epoch = 385 Step =   28793 loss = 0.002
Epoch = 387 Step =   28803 loss = 0.002
Epoch = 389 Step =   28813 loss = 0.002
Epoch = 391 Step =   28823 loss = 0.002
Epoch = 393 Step =   28833 loss = 0.002
Epoch = 395 Step =   28843 loss = 0.002
Epoch = 397 Step =   28853 loss = 0.002
Epoch = 399 Step =   28863 loss = 0.002
Epoch = 401 Step =   28873 loss = 0.002
Epoch = 403 Step =   28883 loss = 0.002


Epoch = 762 Step =   30673 loss = 0.002
Epoch = 764 Step =   30683 loss = 0.002
Epoch = 766 Step =   30693 loss = 0.002
Epoch = 768 Step =   30703 loss = 0.002
Epoch = 770 Step =   30713 loss = 0.002
Epoch = 772 Step =   30723 loss = 0.002
Epoch = 774 Step =   30733 loss = 0.002
Epoch = 776 Step =   30743 loss = 0.002
Epoch = 778 Step =   30753 loss = 0.002
Epoch = 780 Step =   30763 loss = 0.002
Epoch = 782 Step =   30773 loss = 0.002
Epoch = 784 Step =   30783 loss = 0.002
Epoch = 786 Step =   30793 loss = 0.002
Epoch = 788 Step =   30803 loss = 0.002
Epoch = 790 Step =   30813 loss = 0.002
Epoch = 792 Step =   30823 loss = 0.002
Epoch = 794 Step =   30833 loss = 0.002
Epoch = 796 Step =   30843 loss = 0.002
Epoch = 798 Step =   30853 loss = 0.002
Epoch = 800 Step =   30863 loss = 0.002
Epoch = 802 Step =   30873 loss = 0.002
Epoch = 804 Step =   30883 loss = 0.002
Epoch = 806 Step =   30893 loss = 0.002
Epoch = 808 Step =   30903 loss = 0.002
Epoch = 810 Step =   30913 loss = 0.002


Epoch = 1161 Step =   32663 loss = 0.001
Epoch = 1163 Step =   32673 loss = 0.001
Epoch = 1165 Step =   32683 loss = 0.001
Epoch = 1167 Step =   32693 loss = 0.001
Epoch = 1169 Step =   32703 loss = 0.001
Epoch = 1171 Step =   32713 loss = 0.001
Epoch = 1173 Step =   32723 loss = 0.001
Epoch = 1175 Step =   32733 loss = 0.001
Epoch = 1177 Step =   32743 loss = 0.001
Epoch = 1179 Step =   32753 loss = 0.001
Epoch = 1181 Step =   32763 loss = 0.001
Epoch = 1183 Step =   32773 loss = 0.001
Epoch = 1185 Step =   32783 loss = 0.001
Epoch = 1187 Step =   32793 loss = 0.001
Epoch = 1189 Step =   32803 loss = 0.001
Epoch = 1191 Step =   32813 loss = 0.001
Epoch = 1193 Step =   32823 loss = 0.001
Epoch = 1195 Step =   32833 loss = 0.001
Epoch = 1197 Step =   32843 loss = 0.001
Epoch = 1199 Step =   32853 loss = 0.001
Epoch = 1201 Step =   32863 loss = 0.001
Epoch = 1203 Step =   32873 loss = 0.001
Epoch = 1205 Step =   32883 loss = 0.001
Epoch = 1207 Step =   32893 loss = 0.001
Epoch = 1209 Ste

Epoch = 1558 Step =   34643 loss = 0.001
Epoch = 1560 Step =   34653 loss = 0.001
Epoch = 1562 Step =   34663 loss = 0.001
Epoch = 1564 Step =   34673 loss = 0.001
Epoch = 1566 Step =   34683 loss = 0.001
Epoch = 1568 Step =   34693 loss = 0.001
Epoch = 1570 Step =   34703 loss = 0.001
Epoch = 1572 Step =   34713 loss = 0.001
Epoch = 1574 Step =   34723 loss = 0.001
Epoch = 1576 Step =   34733 loss = 0.001
Epoch = 1578 Step =   34743 loss = 0.001
Epoch = 1580 Step =   34753 loss = 0.001
Epoch = 1582 Step =   34763 loss = 0.001
Epoch = 1584 Step =   34773 loss = 0.001
Epoch = 1586 Step =   34783 loss = 0.001
Epoch = 1588 Step =   34793 loss = 0.001
Epoch = 1590 Step =   34803 loss = 0.001
Epoch = 1592 Step =   34813 loss = 0.001
Epoch = 1594 Step =   34823 loss = 0.001
Epoch = 1596 Step =   34833 loss = 0.001
Epoch = 1598 Step =   34843 loss = 0.001
Epoch = 1600 Step =   34853 loss = 0.001
Epoch = 1602 Step =   34863 loss = 0.001
Epoch = 1604 Step =   34873 loss = 0.001
Epoch = 1606 Ste

Epoch = 1952 Step =   36613 loss = 0.001
Epoch = 1954 Step =   36623 loss = 0.001
Epoch = 1956 Step =   36633 loss = 0.001
Epoch = 1958 Step =   36643 loss = 0.001
Epoch = 1960 Step =   36653 loss = 0.001
Epoch = 1962 Step =   36663 loss = 0.001
Epoch = 1964 Step =   36673 loss = 0.001
Epoch = 1966 Step =   36683 loss = 0.001
Epoch = 1968 Step =   36693 loss = 0.001
Epoch = 1970 Step =   36703 loss = 0.001
Epoch = 1972 Step =   36713 loss = 0.001
Epoch = 1974 Step =   36723 loss = 0.001
Epoch = 1976 Step =   36733 loss = 0.001
Epoch = 1978 Step =   36743 loss = 0.001
Epoch = 1980 Step =   36753 loss = 0.001
Epoch = 1982 Step =   36763 loss = 0.001
Epoch = 1984 Step =   36773 loss = 0.001
Epoch = 1986 Step =   36783 loss = 0.001
Epoch = 1988 Step =   36793 loss = 0.001
Epoch = 1990 Step =   36803 loss = 0.001
Epoch = 1992 Step =   36813 loss = 0.001
Epoch = 1994 Step =   36823 loss = 0.001
Epoch = 1996 Step =   36833 loss = 0.001
Epoch = 1998 Step =   36843 loss = 0.001
Epoch = 2000 Ste

Epoch = 2353 Step =   38613 loss = 0.001
Epoch = 2355 Step =   38623 loss = 0.001
Epoch = 2357 Step =   38633 loss = 0.001
Epoch = 2359 Step =   38643 loss = 0.001
Epoch = 2361 Step =   38653 loss = 0.001
Epoch = 2363 Step =   38663 loss = 0.001
Epoch = 2365 Step =   38673 loss = 0.001
Epoch = 2367 Step =   38683 loss = 0.001
Epoch = 2369 Step =   38693 loss = 0.001
Epoch = 2371 Step =   38703 loss = 0.001
Epoch = 2373 Step =   38713 loss = 0.001
Epoch = 2375 Step =   38723 loss = 0.001
Epoch = 2377 Step =   38733 loss = 0.001
Epoch = 2379 Step =   38743 loss = 0.001
Epoch = 2381 Step =   38753 loss = 0.001
Epoch = 2383 Step =   38763 loss = 0.001
Epoch = 2385 Step =   38773 loss = 0.001
Epoch = 2387 Step =   38783 loss = 0.001
Epoch = 2389 Step =   38793 loss = 0.001
Epoch = 2391 Step =   38803 loss = 0.001
Epoch = 2393 Step =   38813 loss = 0.001
Epoch = 2395 Step =   38823 loss = 0.001
Epoch = 2397 Step =   38833 loss = 0.001
Epoch = 2399 Step =   38843 loss = 0.001
Epoch = 2401 Ste

Epoch = 2752 Step =   40603 loss = 0.001
Epoch = 2754 Step =   40613 loss = 0.001
Epoch = 2756 Step =   40623 loss = 0.001
Epoch = 2758 Step =   40633 loss = 0.001
Epoch = 2760 Step =   40643 loss = 0.001
Epoch = 2762 Step =   40653 loss = 0.001
Epoch = 2764 Step =   40663 loss = 0.001
Epoch = 2766 Step =   40673 loss = 0.001
Epoch = 2768 Step =   40683 loss = 0.001
Epoch = 2770 Step =   40693 loss = 0.001
Epoch = 2772 Step =   40703 loss = 0.001
Epoch = 2774 Step =   40713 loss = 0.001
Epoch = 2776 Step =   40723 loss = 0.001
Epoch = 2778 Step =   40733 loss = 0.001
Epoch = 2780 Step =   40743 loss = 0.001
Epoch = 2782 Step =   40753 loss = 0.001
Epoch = 2784 Step =   40763 loss = 0.001
Epoch = 2786 Step =   40773 loss = 0.001
Epoch = 2788 Step =   40783 loss = 0.001
Epoch = 2790 Step =   40793 loss = 0.001
Epoch = 2792 Step =   40803 loss = 0.001
Epoch = 2794 Step =   40813 loss = 0.001
Epoch = 2796 Step =   40823 loss = 0.001
Epoch = 2798 Step =   40833 loss = 0.001
Epoch = 2800 Ste

In [11]:
predict(token_vocab, target_vocab, "맛은 괜찮지만 서비스가 별로에요")

0.992915
POS 0


In [12]:
predict(token_vocab, target_vocab, "재료가 정말 신선하고 좋아요")

1.0
POS 1


In [13]:
predict(token_vocab, target_vocab, "너무 더워서 찝찝해")
predict(token_vocab, target_vocab, "지구는 둥글다")

1.0
POS 0
0.999628
OBJ 0


In [14]:
predict(token_vocab, target_vocab, "회의 결과는 성공적이었습니다")

0.999876
POS 0


In [15]:
predict(token_vocab, target_vocab, "욕실도 널찍하고, 아메니티도 충실히 구비되어 있습니다.")
predict(token_vocab, target_vocab, "집을 떠나 왔지만, 집처럼 편한 곳입니다.")
predict(token_vocab, target_vocab, "일단 아침식사 메뉴가 풍부해서 놀랐습니다.")
predict(token_vocab, target_vocab, "윗층은 옆 건물이 낮아서 전망이 좋습니다.")
predict(token_vocab, target_vocab, "와이프와 저는 파리에 있는 이 아담하고 고급스런 호텔에서 며칠을 보냈습니다.")
predict(token_vocab, target_vocab, "일부러 편리한 곳을 찾아서 발견한 비지니스 호텔은 아니었지만 실제로 가야바쵸 역에서도 가까워 편리했습니다.")

1.0
POS 0
1.0
POS 1
0.999869
POS 0
0.994909
NEG 0
0.99915
POS 0
0.999994
POS 0


In [27]:
predict(token_vocab, target_vocab, "레스토랑이 너무음식 가지러 가기 힘들었습니다. 넓어서 ")

0.999999
NEG 0
